In [1]:
cd ../..

e:\GitHub\computervision


In [2]:
import numpy as np
import os
import torch

import torch.nn as nn
import torch.optim as optim

from torchvision import models, transforms
from modules.ai_util import PytorchDataset, PytorchTraining

In [3]:
# Variables for continuing previous training
CONTINUE_TRAINING = False
MODEL_WEIGHTS_PATH = 'notebooks/vit/models/01-24-2024 19-25-41/'
MODEL_LAST_EPOCH = 'epoch_8.pth'

# CONFIG VARIABLES
DATA_DIR = 'E:\Datasets\cor-splits\sgkf-8-1-1-4000'
START_EPOCH = 1 # Change it if you're continuing an existing run
N_EPOCHS = 31
LR_EPOCH_SCHEDULE = 10
LR_GAMMA = 0.1
SAVE_INTERVAL = 5 # How many epochs between each backup

In [4]:
working_folder = 'notebooks/efficientnet'
models_folder = 'models'
from datetime import datetime

curr_time = datetime.now()
curr_time = curr_time.strftime("%Y-%m-%d %H-%M-%S")

output_folder = os.path.join(working_folder, models_folder)
output_folder = os.path.join(output_folder, curr_time)

In [5]:
# efficientnet_b0
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.Resize(256, transforms.InterpolationMode.BICUBIC),
        transforms.CenterCrop(224),
        transforms.ToTensor(), # When converting to tensor, pytorch automatically rescales to [0,1]
        transforms.Normalize(mean, std)
    ]),
    'val': transforms.Compose([
        transforms.Resize(256, transforms.InterpolationMode.BICUBIC),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
}

In [6]:
model = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.DEFAULT)

for param in model.parameters(): # Turning parameters not trainable
    param.requires_grad = True

# A newly defined layer is created with requires_grad=True by default
model.classifier = nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(in_features=1280, out_features=2)
    )

if CONTINUE_TRAINING:
    output_folder = MODEL_WEIGHTS_PATH
    model.load_state_dict(torch.load(output_folder+MODEL_LAST_EPOCH))
else:
    if not os.path.exists(output_folder): 
        os.makedirs(output_folder)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
             
model = model.to(device)

criterion = nn.CrossEntropyLoss()

# Only the fully connected layer is being optimized
optimizer = optim.SGD(model.classifier.parameters(), lr=0.001)

# StepLR Decays the learning rate of each parameter group by gamma every step_size epochs
# Learning rate scheduling should be applied after optimizer’s update
# e.g., you should write your code this way:
# for epoch in range(100):
#     train(...)
#     validate(...)
#     scheduler.step()
step_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=LR_EPOCH_SCHEDULE, gamma=LR_GAMMA)

dataset = PytorchDataset(DATA_DIR, data_transforms)
trainer = PytorchTraining(device, dataset, output_folder)

model = trainer.train_pytorch_model(model, criterion, optimizer, step_lr_scheduler, START_EPOCH, N_EPOCHS, SAVE_INTERVAL)

Epoch 1/31
----------
train Loss: 0.6750 Acc: 0.5811
val Loss: 0.6658 Acc: 0.6241
Epoch duration: 1 m 37s
Learning Rate = 0.001

Epoch 2/31
----------
train Loss: 0.6396 Acc: 0.6707
val Loss: 0.6363 Acc: 0.7578
Epoch duration: 1 m 39s
Learning Rate = 0.001

Epoch 3/31
----------
train Loss: 0.6144 Acc: 0.7078
val Loss: 0.6233 Acc: 0.7780
Epoch duration: 1 m 39s
Learning Rate = 0.001

Epoch 4/31
----------
train Loss: 0.5904 Acc: 0.7420
val Loss: 0.6044 Acc: 0.7983
Epoch duration: 1 m 39s
Learning Rate = 0.001

Epoch 5/31
----------
train Loss: 0.5750 Acc: 0.7541
val Loss: 0.6021 Acc: 0.7470
Epoch duration: 1 m 39s
Learning Rate = 0.001

Epoch 6/31
----------
train Loss: 0.5538 Acc: 0.7697
val Loss: 0.5866 Acc: 0.7589
Epoch duration: 1 m 40s
Learning Rate = 0.001

Epoch 7/31
----------
train Loss: 0.5386 Acc: 0.7797
val Loss: 0.5789 Acc: 0.7828
Epoch duration: 1 m 39s
Learning Rate = 0.001

Epoch 8/31
----------
train Loss: 0.5258 Acc: 0.7852
val Loss: 0.5748 Acc: 0.7542
Epoch duration: